# Connectomes Data and Modeling Techniques

### Techniques covered in this Notebook
- Preprocessing the functional connectome data
- RiemannianMinimumDistanceToMean classification algorithm

## Functional Connectomes

A **functional connectome** refers to the theoretical concept of the brain's functional connections, which describes how different brain regions interact and communicate with each other.

**Functional connectome data**, on the other hand, is the actual data that represents these connections, typically obtained through neuroimaging techniques such as functional MRI. This data provides a quantitative measure of the synchronized activity between brain regions, allowing researchers to study and analyze the functional connectome.

This data is crucial in ADHD research, as it can help us understand how brain connectivity patterns differ between individuals with ADHD and those without. By analyzing these patterns, we can identify potential biomarkers for ADHD and develop more accurate diagnostic tools.


## The 2025 Datathon Functional Connectome Data

The dataset correponds to the Functional Connectivity Networks (FCN) extracted from resting-state fMRIs of **1213 patients at 200 Regions Of Interest (ROIs)**. Patients are separated in two classes: ADHD and control. The goal will be to classify them. (You can also use these techniques to classify the sex of the patients.)







## Functional Connectome Data and SPD Matrices

Functional connectome data is often approximated as a Symmetric Positive Definite (SPD) matrix. However, in reality, functional connectome data may not always be perfectly SPD.

# Load the data

In [21]:
!pip install geomstats

In [ ]:
#Mount Google Drive (only needed when run on Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import csv
import openpyxl
import matplotlib.pyplot as plt

import geomstats.datasets.utils as data_utils
import geomstats.backend as gs
from geomstats.geometry.skew_symmetric_matrices import SkewSymmetricMatrices
import time

# Start time measurement:
start_time = time.time()

# Read in the data (update to your root folder)
df_soln = pd.read_excel("/content/drive/MyDrive/WIDS_2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")
df_conn = pd.read_csv("/content/drive/MyDrive/WIDS_2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
df_conn_test = pd.read_csv("/content/drive/MyDrive/WIDS_2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")

# Extract the ADHD solutions and sort the data by participant_id
df_soln_adhd = df_soln[['participant_id', 'ADHD_Outcome']].sort_values('participant_id')
df_conn = df_conn.sort_values('participant_id')

### Reshape the connectome data into symmetric matrices

We are given the upper half of the connectome matrices as vectors, which represent the functional connections between different brain regions. However, to analyze and process this data using Riemannian geometry-based methods, we need to reshape it into symmetric matrices.

By reshaping the upper half vectors into symmetric matrices, we can reconstruct the full matrix, which is a more natural representation of the brain's functional connectivity.

In [24]:
# Define the load_connectomes function
def load_connectomes(df_conn, df_soln_adhd, as_vectors=False):
    """
    Load brain connectome data and ADHD labels, returning symmetric matrices with ones on the diagonal.
    """

    patient_id = gs.array(df_conn['participant_id'])
    data = gs.array(df_conn.drop('participant_id', axis=1))
    target = gs.array(df_soln_adhd['ADHD_Outcome'])

    if as_vectors:
        return data, patient_id, target
    mat = SkewSymmetricMatrices(200).matrix_representation(data)
    mat = gs.eye(200) - gs.transpose(gs.tril(mat), (0, 2, 1))
    mat = 1.0 / 2.0 * (mat + gs.transpose(mat, (0, 2, 1)))

    return mat, patient_id, target

In [ ]:
# Call the load_connectomes function
data, patient_id, labels = load_connectomes(df_conn, df_soln_adhd)

# Print the results
print(f"There are {len(data)} connectomes: {sum(labels==0)} non-ADHD and {sum(labels==1)} ADHD patients.")

In [ ]:
data.shape

We now have 200 x 200 matrices for each of the 1213 patients

## Checking for SPD Manifold Membership

Check if the connectome data lies on the Symmetric Positive Definite (SPD) manifold. We use the SPDMatrices class from the geomstats library to check for SPD property.

In [ ]:
from geomstats.geometry.spd_matrices import SPDMatrices

manifold = SPDMatrices(200, equip=False)
print(gs.all(manifold.belongs(data)))

In [ ]:
# Count the number of connectomes that do not lie on the SPD manifold

count_false = np.sum(~(manifold.belongs(data)))
print("Count of False:", count_false)

### Ensuring SPD Property

To ensure the data is Symmetric Positive Definite (SPD), we can add a small diagonal matrix to the original data. This approach modifies the data minimally while guaranteeing the SPD property. The small diagonal matrix is added to each 2D slice of the 3D matrix, but the correction is only non-zero for the slices that are not SPD.

In [ ]:
# Function to add a diagonal matrix to a 2D matrix
def add_diagonal_correction(matrix):
    eigenvalues = np.linalg.eigvals(matrix)
    min_eigenvalue = np.min(eigenvalues)

    if min_eigenvalue < 0:
        correction = -min_eigenvalue + 1e-6
        correction_matrix = correction * np.eye(matrix.shape[0])
        return matrix + correction_matrix
    else:
        return matrix

# Apply the correction to each 2D slice of the 3D matrix
data_corrected = np.array([add_diagonal_correction(slice) for slice in data])

print("Original Matrix shape:", data.shape)
print("Corrected Matrix shape:", data_corrected.shape)

print(gs.all(manifold.belongs(data_corrected)))

#### Counting differences in original data and corrected data

We expect the count of differences to be 12 X 200 = 2400, since we added a correction to 12 connectomes, each with 200 features.

In [ ]:
def count_differences(array1, array2, tolerance=1e-6):
    """
    This function compares two 3D arrays and returns the count of differences.
    """
    if array1.shape != array2.shape:
        raise ValueError("Arrays must be of the same shape")

    differences = np.greater(np.abs(array1 - array2), tolerance)
    count = np.sum(differences)

    return count

print(count_differences(data, data_corrected))

# Classification algorithm: RiemannianMinimumDistanceToMean

**Reference** Geometric Approaches for Processing Brain Connectomes video: https://www.youtube.com/watch?v=vtHBOBOcn6E

The RiemannianMinimumDistanceToMean algorithm is based on Riemannian geometry, which is a mathematical framework that allows us to analyze and process data on curved spaces, such as the space of symmetric positive definite (SPD) matrices. This is particularly useful for brain connectome data, which can be represented as SPD matrices. This algorithm calculates the Riemannian distance between each connectome and the mean of each class, and assigns the connectome to the class with the smallest distance.

### Define the model

In [ ]:
from geomstats.learning.mdm import RiemannianMinimumDistanceToMean

spd_manifold = SPDMatrices(n=200, equip=True)
mdm = RiemannianMinimumDistanceToMean(space=spd_manifold)

### Split data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split
X = data_corrected; y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=47)

### Print data statistics

We examine the class distribution in the full dataset, as well as the train and test sets, to ensure that they are similar and representative of the overall data. This is crucial for training a reliable model, as a skewed class distribution can lead to biased results.

In [ ]:
print(f"The dataset has {len(X)} connectomes.")
print(f"The train set has {len(X_train)} connectomes and has size {X_train.shape}.")
print(f"The test set has {len(X_test)} connectomes and has size {X_test.shape}.")

print("Full dataset class distribution:")
print(pd.Series(y).value_counts(normalize=True) * 100)

print("\nTrain dataset class distribution:")
print(pd.Series(y_train).value_counts(normalize=True) * 100)

print("\nTest dataset class distribution:")
print(pd.Series(y_test).value_counts(normalize=True) * 100)

### Train and Evaluate the model

In [ ]:
mdm.fit(X_train, y_train)
print(mdm.score(X_test, y_test))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
y_pred = mdm.predict(X_test)
print("F1 score:", f1_score(y_test, y_pred))

In [ ]:
y_train_pred = mdm.predict(X)
print("F1 score:", f1_score(y, y_train_pred))

In [ ]:
from scipy.special import expit
y_train_probabilities = expit(y_train_pred)

In [ ]:
patient_id_np = np.array(patient_id)
print('Patient id np shape: ', patient_id_np.shape)
print('Y train probs shape: ', y_train_probabilities.shape)


In [ ]:
y_train_probabilities_df = pd.DataFrame({'participant_id': patient_id_np, 'ADHD_pred': y_train_pred, 'ADHD_prob': y_train_probabilities})
y_train_probabilities_df.head()

In [ ]:
y_train_probabilities_df.to_csv('/content/drive/MyDrive/WIDS_2025/Output/mdm_train_pred_prob_adhd.csv', index=False)

## Prediction on the test set

In [ ]:
def load_connectomes_test(df_conn):

    """
    Load brain connectome data, returning symmetric matrices with ones on the diagonal.
    """

    patient_id = gs.array(df_conn['participant_id'])
    data = gs.array(df_conn.drop('participant_id', axis=1))

    mat = SkewSymmetricMatrices(200).matrix_representation(data)
    mat = gs.eye(200) - gs.transpose(gs.tril(mat), (0, 2, 1))
    mat = 1.0 / 2.0 * (mat + gs.transpose(mat, (0, 2, 1)))

    return mat, patient_id

In [ ]:
data_test, patient_id_test = load_connectomes_test(df_conn_test)

In [ ]:
data_test.shape

In [ ]:
manifold = SPDMatrices(200, equip=False)
print(gs.all(manifold.belongs(data_test)))

In [ ]:
# Count the number of connectomes that do not lie on the SPD manifold

count_false = np.sum(~(manifold.belongs(data_test)))
print("Count of False:", count_false)

In [ ]:
# Apply the correction to each 2D slice of the 3D matrix
data_test_corrected = np.array([add_diagonal_correction(slice) for slice in data_test])

print("Original Matrix shape:", data_test.shape)
print("Corrected Matrix shape:", data_test_corrected.shape)

print(gs.all(manifold.belongs(data_test_corrected)))

In [ ]:
# Count differences between original and corrected test data:
print(count_differences(data_test, data_test_corrected))

In [ ]:
y_pred_test = mdm.predict(data_test_corrected)

In [ ]:
y_pred_test_probabilities = expit(y_pred_test)

In [ ]:
patient_id_test_np = np.array(patient_id_test)

In [ ]:
y_pred_test_probabilities_df = pd.DataFrame({'participant_id': patient_id_test_np, 'ADHD_pred': y_pred_test, 'ADHD_prob': y_pred_test_probabilities})
y_pred_test_probabilities_df.head()

In [ ]:
y_pred_test_probabilities_df.to_csv('/content/drive/MyDrive/WIDS_2025/Output/mdm_test_pred_prob_adhd.csv', index=False)

In [ ]:
submission_mdm = pd.DataFrame({'participant_id': patient_id_test_np, 'ADHD_pred': y_pred_test})
submission_mdm.to_csv('/content/drive/MyDrive/WIDS_2025/Output/submission_mdm_adhd.csv', index=False)

In [ ]:
# Print runtime
end_time = time.time()
runtime = round((end_time - start_time)/60, 1)
print('Runtime of ADHD prediction with MDM: ', runtime, 'minutes')